<a href="https://colab.research.google.com/github/LanaSina/DynamicChangeBlindness/blob/master/chainer_prednet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install tb chainer

Mount drive (fpsi images are in .zip file inside drive)


In [0]:
# check python version (this should be python 2)
# change python version through menu: Runtime -> Change runtime type
# choose python 2 and gpu
import sys
print(sys.version)


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%pip install tensorboard-chainer

Import prednet files from github

Load files and locally unzip images

Create image lists

In [0]:
 !rm -r chainer_prednet
 #!rm -r images
  
 !git clone --branch benham_speed https://github.com/LanaSina/chainer_prednet.git
  

In [0]:
%load chainer_prednet/PredNet/net.py
%load chainer_prednet/PredNet/main.py



In [0]:
!rm -r input_images

In [0]:
# unzip images
!mkdir 'expanding'
# replace with the path to your drive (sometimes is gdrive rather than drive)
!unzip -q 'drive/My Drive/AI/prednet_private/expanding.zip' -d 'expanding'
!mv "expanding/expanding" "expanding/input_images"


In [0]:
# unzip images
!mkdir 'fpsi'
# replace with the path to your drive (sometimes is gdrive rather than drive)
!unzip -q 'drive/My Drive/AI/prednet_private/input_images.zip' -d 'fpsi/'


In [0]:
# create black and white input
%run 'chainer_prednet/convert_to_greyscale.py' 'fpsi/input_images' '5000'

In [0]:
# just build text files
#%mv "fpsi/images" "fpsi/input_images"
%run 'chainer_prednet/generate_imagelist.py' 'greyscale_input/' '5000'
#'drive/My Drive/AI/prednet_private/lines_forward_5'

In [0]:
%run 'chainer_prednet/PredNet/main.py' --images 'greyscale_input' -g 0 --period 500000 --input_len 5000

In [0]:
!cp 'drive/My Drive/AI/prednet_private/500000.model' '500000.model'

In [0]:
!rm -r "result"
#%run 'chainer_prednet/PredNet/main.py' --images 'drive/My Drive/AI/prednet_private/lines_black_bg' --initmodel '500000.model' --test
%run 'chainer_prednet/PredNet/main.py' --images 'greyscale_input' --initmodel '500000.model' --input_len 5000 --test


In [0]:
!zip -r "result_images.zip" "result" 
from google.colab import files
files.download('result_images.zip') 


In [0]:
run "chainer_prednet/batch_rename.py" "result/"

In [0]:
run "chainer_prednet/verify_predicted_colors.py" "result/" "fpsi/input_images/"  "greyscale_input/input_images" "greyscale_analysis"


In [0]:
# !apt install ffmpeg

#!ffmpeg -framerate 10 -pattern_type glob -i "greyscale_analysis/col_diff/*.png" 'greyscale_analysis/col_diff/_out.mp4'
#!ffmpeg -framerate 10 -pattern_type glob -i "greyscale_analysis/col_match/*.png" 'greyscale_analysis/col_match/_out.mp4'
#!ffmpeg -framerate 10 -pattern_type glob -i "greyscale_analysis/col_overlay/*.png" 'greyscale_analysis/col_overlay/_out.mp4'
!ffmpeg -framerate 10 -pattern_type glob -i "result/*.png" 'result/_out.mp4'

In [0]:
#files.download('greyscale_analysis/col_diff/_out.mp4') 
#files.download('greyscale_analysis/col_match/_out.mp4') 
#files.download('greyscale_analysis/col_overlay/_out.mp4') 
files.download('result/_out.mp4')


In [0]:
# image analysis
%run "chainer_prednet/images_analysis.py" -i "fpsi/input_images"